# Terra Signal Hackathon: Grupo 2

In [0]:
%pip install pandas
%pip install xgboost

In [0]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    classification_report, 
    roc_auc_score, 
    confusion_matrix
)
from sklearn.metrics import RocCurveDisplay
from sklearn.dummy import DummyClassifier
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

# Read the CSV file using pandas
file_path = "./history.csv"
df = pd.read_csv(file_path)
df.head().transpose()

##Análise Exploratória e limpeza dos dados

In [0]:
df.info()

In [0]:
for col in df.columns:
    print(f"\n--- {col} ---")
    print(df[col].value_counts())


In [0]:
def clean(df):
    # criando um data frame limpo
    df_clean = df.copy()
    df_clean.columns = df_clean.columns.str.strip()

    # limpando a coluna 'tenure'

    # substituindo 'unknown' por NaN e convertendo para numérico
    df_clean["tenure"] = df_clean["tenure"].replace("unknown", pd.NA)
    df_clean["tenure"] = pd.to_numeric(df_clean["tenure"], errors="coerce")

    # substituindo valores 0 por NaN
    df_clean.loc[df_clean["tenure"] == 0, "tenure"] = pd.NA

    # preenchendo NaN com a mediana e convertendo para inteiro
    df_clean["tenure"] = df_clean["tenure"].fillna(df_clean["tenure"].median())
    df_clean["tenure"] = df_clean["tenure"].astype(int)

    # normalizando phone service
    df_clean["PhoneService"] = (
        df_clean["PhoneService"]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace({"yes": 1, "no": 0})
        .astype(int)  # <- evitar FutureWarning
    )

    # normalizando multiple lines
    df_clean["MultipleLines"] = (
        df_clean["MultipleLines"]
        .replace({"No phone service": "No"})
        .map({"Yes": 1, "No": 0})
        .astype(int)
    )

    # normalizando colunas de internet
    internet_cols = [
        "OnlineSecurity", "OnlineBackup", "DeviceProtection",
        "TechSupport", "StreamingTV", "StreamingMovies"
    ]

    for col in internet_cols:
        df_clean[col] = (
            df_clean[col]
            .replace({"No internet service": "No"})
            .map({"Yes": 1, "No": 0})
            .astype(int)
        )

    # normalizando colunas binárias
    for col in ["Partner", "Dependents", "PaperlessBilling"]:
        df_clean[col] = df_clean[col].map({"Yes": 1, "No": 0}).astype(int)

    # convertendo total charges para numérico e tratando NaN
    df_clean["TotalCharges"] = pd.to_numeric(df_clean["TotalCharges"], errors="coerce")
    df_clean["TotalCharges"] = df_clean["TotalCharges"].fillna(df_clean["TotalCharges"].median())

    # limpando coluna de feedback do cliente
    df_clean["CustomerFeedback"] = df_clean["CustomerFeedback"].fillna("").astype(str)
    df_clean["CustomerFeedback_clean"] = (
        df_clean["CustomerFeedback"]
        .str.lower()
        .str.replace("[^a-zA-Z0-9 ]", "", regex=True)
    )

    # tratando categóricas com one-hot encoding
    cat_cols = [
        "gender", "InternetService", "Contract", "PaymentMethod"
    ]

    df_clean = pd.get_dummies(df_clean, columns=cat_cols, drop_first=True)

    # convertendo target para binário
    if "Churn" in df_clean.columns:
        df_clean["Churn"] = df_clean["Churn"].map({"Yes": 1, "No": 0}).astype(int)
        
    return df_clean

df_clean = clean(df)


In [0]:
df_clean.info()


In [0]:
df_clean.head()

In [0]:
df_clean.to_csv("history_clean.csv", index=False)

In [0]:
df_clean.columns.tolist()


In [0]:
# Selecionar apenas as 3 colunas que serão usadas no classificador de IA dos motivos
df_minimal = df_clean[["customerID", "Churn", "CustomerFeedback_clean"]]

# Converter para Spark DataFrame
df_spark = spark.createDataFrame(df_minimal)
df_spark.createOrReplaceTempView("tmp_history_clean")


In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn.history_clean AS
SELECT customerID, Churn, CustomerFeedback_clean
FROM tmp_history_clean;

In [0]:
plt.figure(figsize=(6,4))
sns.countplot(
    data=df_clean,
    x="Churn",
    hue="Churn",
    palette="Set2",
    legend=False
)
plt.title("Quantidade de Clientes: Churn vs Não Churn")
plt.xticks([0,1], ["Não churn", "Churn"])
plt.ylabel("Quantidade")
plt.xlabel("")
plt.show()


In [0]:
plt.figure(figsize=(10,5))
sns.histplot(
    data=df_clean,
    x="tenure",
    hue="Churn",
    multiple="stack",
    bins=40,
    palette="Set2"
)
plt.title("Distribuição de Tenure por Churn")
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.boxplot(
    data=df_clean,
    x="Churn",
    y="MonthlyCharges",
    hue="Churn",
    palette="Set2",
    legend=False   # evita legenda duplicada
)
plt.title("MonthlyCharges por Churn")
plt.xticks([0,1], ["Não churn", "Churn"])
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.violinplot(data=df_clean, x="Churn", y="TotalCharges", hue="Churn", palette="Set2", legend=False)
plt.title("Total Charges por Churn")
plt.show()


In [0]:
colors = ["#C0C0C0", "#2ECC71"]
# Reconstroi Contract
Contract_series = df_clean.apply(
    lambda r: (
        "Two year" if r["Contract_Two year"] == 1 else
        "One year" if r["Contract_One year"] == 1 else
        "Month-to-month"
    ),
    axis=1
)

# Reconstroi InternetService
InternetService_series = df_clean.apply(
    lambda r: (
        "Fiber optic" if r["InternetService_Fiber optic"] == 1 else
        "No internet" if r["InternetService_No"] == 1 else
        "DSL"
    ),
    axis=1
)

# Reconstroi PaymentMethod
PaymentMethod_series = df_clean.apply(
    lambda r: (
        "Credit card (automatic)" if r["PaymentMethod_Credit card (automatic)"] == 1 else
        "Electronic check" if r["PaymentMethod_Electronic check"] == 1 else
        "Mailed check" if r["PaymentMethod_Mailed check"] == 1 else
        "Bank transfer (automatic)"
    ),
    axis=1
)


In [0]:
def plot_stacked(series, churn, title, xlabel):
    ct = pd.crosstab(series, churn, normalize="index")
    ct.columns = ["No", "Yes"]
    ct.plot(kind="bar", stacked=True, figsize=(7,4), color=colors)
    plt.title(title)
    plt.ylabel("Proporção")
    plt.xlabel(xlabel)
    plt.ylim(0,1)
    plt.legend(title="Churn")
    plt.show()


In [0]:
plot_stacked(
    Contract_series,
    df_clean["Churn"],
    "Composição: Churn vs Não Churn — Contract",
    "Contract Type"
)

plot_stacked(
    InternetService_series,
    df_clean["Churn"],
    "Composição: Churn vs Não Churn — Internet Service",
    "Internet Service Type"
)
plot_stacked(
    PaymentMethod_series,
    df_clean["Churn"],
    "Composição: Churn vs Não Churn — Payment Method",
    "Payment Method"
)


In [0]:
plt.figure(figsize=(14,10))
corr = df_clean.corr(numeric_only=True)
sns.heatmap(corr, annot=False, cmap="coolwarm", linewidths=.5)
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.show()


In [0]:
ct_os = pd.crosstab(
    df_clean["OnlineSecurity"],
    df_clean["Churn"],
    normalize='index'
)
ct_os.columns = ["No", "Yes"]

ct_os.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn vs Não Churn — OnlineSecurity (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("OnlineSecurity")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
ct_ts = pd.crosstab(
    df_clean["TechSupport"],
    df_clean["Churn"],
    normalize='index'
)
ct_ts.columns = ["No", "Yes"]

ct_ts.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn vs Não Churn — TechSupport (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("TechSupport")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
ct_stv = pd.crosstab(
    df_clean["StreamingTV"],
    df_clean["Churn"],
    normalize='index'
)
ct_stv.columns = ["No", "Yes"]

ct_stv.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn vs Não Churn — StreamingTV (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("StreamingTV")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
ct_pb = pd.crosstab(
    df_clean["PaperlessBilling"],
    df_clean["Churn"],
    normalize='index'
)
ct_pb.columns = ["No", "Yes"]

ct_pb.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: Churn — Paperless Billing (0 = No, 1 = Yes)")
plt.ylabel("Proporção")
plt.xlabel("Paperless Billing")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
plt.figure(figsize=(7,4))
sns.boxplot(x=InternetService_series, y=df_clean["MonthlyCharges"])
plt.title("Distribuição de MonthlyCharges por Tipo de Internet")
plt.xlabel("Internet Service Type")
plt.ylabel("MonthlyCharges")
plt.show()


plt.figure(figsize=(8,5))
sns.boxplot(x=Contract_series, y=df_clean["MonthlyCharges"])
plt.title("Distribuição de MonthlyCharges por Tipo de Contract")
plt.xlabel("Contract Type")
plt.ylabel("MonthlyCharges")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8,5))
sns.boxplot(x=PaymentMethod_series, y=df_clean["MonthlyCharges"])
plt.title("Distribuição de MonthlyCharges por Payment Method")
plt.xlabel("Payment Method")
plt.ylabel("MonthlyCharges")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()


# Conclusões Completas da Análise Exploratória

## 1. Distribuição geral do churn
O gráfico de contagem mostra que a maioria dos clientes não churnou, com churn representando uma minoria relevante.  
Isso revela um desbalanceamento natural da variável-alvo, mas não impede a análise exploratória.

## 2. Relação entre tenure e churn
O histograma mostra um padrão claro:
- Clientes com tenure muito baixo apresentam alta incidência de churn.
- Conforme o tenure aumenta, a proporção de churn cai drasticamente.
- Clientes com tenure muito alto quase não apresentam churn.

Conclusão: churn é predominantemente concentrado nos clientes que recém entraram na base.

## 3. MonthlyCharges e churn
O boxplot indica:
- Clientes churn possuem MonthlyCharges mais altos.
- A mediana e o intervalo interquartil de churn são superiores aos de não churn.

Conclusão: mensalidades altas aumentam a probabilidade de churn.

## 4. TotalCharges e churn
O violin plot mostra:
- Clientes churn apresentam TotalCharges muito mais baixos.
- Clientes não churn concentram-se em valores altos de TotalCharges.

Conclusão: churn está fortemente associado a clientes com pouco tempo de relacionamento (TotalCharges baixo é um proxy de tenure baixo).

## 5. Tipo de Internet e churn (InternetService reconstruído)
O gráfico de proporção mostra três padrões:
- Fiber optic apresenta a maior proporção de churn.
- DSL tem churn moderado.
- No internet praticamente não tem churn.

Conclusão: o tipo de internet é um driver importante, com fibra óptica associada a maior insatisfação ou maior sensibilidade a preço.

## 6. Método de pagamento e churn
O gráfico de PaymentMethod indica:
- Clientes que utilizam Electronic check possuem proporção significativamente maior de churn.
- Os demais métodos apresentam churn bem menor.

Conclusão: Electronic check é um forte indicador de risco.

## 7. Contrato (Contract reconstruído)
No gráfico reconstruído:
- Month-to-month apresenta claramente a maior proporção de churn.
- One year e Two year exibem churn drasticamente menor.

Conclusão: contrato é um dos fatores de retenção mais relevantes: quanto mais longo, menor o churn.

## 8. OnlineSecurity e churn
O gráfico mostra:
- Clientes sem OnlineSecurity têm churn visivelmente maior.
- Clientes com OnlineSecurity churnam menos.

Conclusão: segurança adicional funciona como um mecanismo de retenção.

## 9. TechSupport e churn
O padrão é semelhante ao anterior:
- Ausência de TechSupport está associada a maior churn.
- Clientes com suporte apresentam taxas menores.

Conclusão: suporte técnico reduz churn, possivelmente por aumentar valor percebido.

## 10. StreamingTV e churn
O gráfico mostra:
- Pequena diferença de churn entre usar ou não usar StreamingTV.
- A variável não apresenta impacto forte.

Conclusão: StreamingTV é um fator secundário e não parece explicar churn de maneira contundente.

## 11. Heatmap de correlação
O heatmap confirma:
- As correlações de Pearson entre as variáveis numéricas e churn são baixas ou nulas.
- Isso é esperado, pois churn é uma variável binária com relações não lineares.

Conclusão: correlação linear não é apropriada para medir relação com churn; os gráficos categóricos capturam muito melhor os padrões.

---

# Síntese Geral dos Principais Fatores de Churn

## Fatores fortes (alta separação nos gráficos)
- Tenure baixo → churn alto.
- MonthlyCharges alto → maior churn.
- TotalCharges baixo → churn alto (clientes novos).
- InternetService = Fiber optic → churn elevado.
- PaymentMethod = Electronic check → churn elevado.
- Contract = Month-to-month → maior churn da base.
- Ausência de OnlineSecurity e TechSupport → aumento do churn.

## Fatores fracos
- StreamingTV (impacto pequeno).
- Outras variáveis numéricas não mostram relação linear significativa.


##Escolha de modelo

In [0]:
# target
y = df_clean["Churn"]

# features (excluindo target e textos)
X = df_clean.drop(columns=["customerID", "Churn", "CustomerFeedback", "CustomerFeedback_clean"])

numeric_cols = X.select_dtypes(include=["int64","float64"]).columns.tolist()
boolean_cols = X.select_dtypes(include=["bool"]).columns.tolist()


preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("bool", "passthrough", boolean_cols)
    ]
)

models = {
    "log_reg": LogisticRegression(max_iter=2000, class_weight="balanced"),
    "random_forest": RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        class_weight="balanced",
        random_state=42
    ),
    "xgboost": XGBClassifier(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        random_state=42
    )
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    pipe = Pipeline([
        ("preprocess", preprocess),
        ("model", model)
    ])
    
    scores = cross_val_score(pipe, X, y, cv=cv, scoring="roc_auc")
    results[name] = scores
    print(f"{name} → AUC médio = {scores.mean():.4f} | std = {scores.std():.4f}")

best_model_name = max(results, key=lambda k: results[k].mean())
best_model = models[best_model_name]

print("Melhor modelo:", best_model_name)

##Certificação

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

pipe_best = Pipeline([
    ("preprocess", preprocess),
    ("model", best_model)
])

pipe_best.fit(X_train, y_train)

y_pred = pipe_best.predict(X_test)\
    
y_proba = pipe_best.predict_proba(X_test)[:,1]


print("=== CERTIFICAÇÃO DO MODELO ===")
print(classification_report(y_test, y_pred, digits=3))
print("AUC:", roc_auc_score(y_test, y_proba))
print("Matriz de confusão:\n", confusion_matrix(y_test, y_pred))

print("=== COMPARAÇÃO COM DUMMY BASELINE ===")
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)

y_dummy = dummy.predict(X_test)
y_dummy_prob = dummy.predict_proba(X_test)[:,1]

print("=== Dummy Classifier (Most Frequent) ===")
print(classification_report(y_test, y_dummy, zero_division=0))
print("AUC:", roc_auc_score(y_test, y_dummy_prob))
print(confusion_matrix(y_test, y_dummy))

In [0]:
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Curva ROC - Modelo Selecionado")
plt.show()

## Treinamento do modelo final com dataset completo

In [0]:
print("=== TREINANDO MODELO FINAL COM DATASET COMPLETO ===")

# Criar pipeline final com os mesmos parâmetros do melhor modelo
pipe_final = Pipeline([
    ("preprocess", preprocess),
    ("model", best_model)
])

# Treinar com 100% dos dados
pipe_final.fit(X, y)

print(f"Modelo final treinado com {len(X)} amostras")
print("Pronto para inferência!")

##Registrando modelo no MLflow Model Registry

In [0]:
# usa o pipeline já treinado no notebook
base_model = pipe_final  # seu pipeline treinado

# exemplo de input para assinatura / input_example
input_example = X.head(3)

# wrapper que retorna DataFrame com prediction e probabilidades
class ProbWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        # usamos o objeto em memória (pipe_final)
        self.model = base_model

    def predict(self, context, model_input):
        # model_input é um DataFrame com as mesmas colunas que seu pipeline espera
        probs = self.model.predict_proba(model_input)  # shape (n, 2)
        preds = self.model.predict(model_input)        # shape (n,)
        # retornar um DataFrame para ficar claro no serving
        out = pd.DataFrame({
            "prediction": preds.astype(int),
            "prob_class0": probs[:, 0],
            "prob_class1": probs[:, 1]
        })
        return out

# nome do registered model no workspace (novo nome para não sobrescrever)
registered_model_name = "workspace_modelo_churn_with_probs"

with mlflow.start_run(run_name="register_pyfunc_probs"):
    mlflow.pyfunc.log_model(
        artifact_path="model_pyfunc_probs",
        python_model=ProbWrapper(),
        registered_model_name=registered_model_name,
        input_example=input_example.head(3),
        signature=infer_signature(input_example.head(3),
                                  base_model.predict_proba(input_example.head(3)))
    )
    mlflow.log_param("note", "pyfunc wrapper that returns prediction + probabilities")

print("Registrado:", registered_model_name)

## Limpeza de inference.csv

In [0]:
inference_raw = pd.read_csv("./inference.csv")
inference_clean = clean(inference_raw)
inference_clean.to_csv("./inference_clean.csv", index=False)


## Aplicação do Modelo Classificador ao dataset inference.csv e geração do Prediction.csv

In [0]:
def prediction_function(input_df):
    # o pipeline já deve receber o dataframe completo e retornar as previsões (0/1)
    preds = pipe_final.predict(input_df)

    # converte 0/1 para Yes/No
    preds = pd.Series(preds).map({1: "Yes", 0: "No"})

    # cria dataframe de saída seguindo o formato anterior
    output = pd.DataFrame({
        'customerID': input_df['customerID'],
        'prediction': preds
    })
    
    return output

# leitura dos dados
test_df = pd.read_csv('inference_clean.csv')

# obtém predições
prediction = prediction_function(test_df)

print(prediction.head().transpose())

# salva arquivo para submissão
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
prediction.to_csv(f'prediction_group2_{timestamp}.csv', index=False)


# Arquivo csv de predictions com probabilidades


In [0]:
df_out = inference_clean.copy()


binary_cols = [c for c in df_out.columns 
               if df_out[c].isin([0,1]).all() and c != "SeniorCitizen"]

for col in binary_cols:
    df_out[col] = df_out[col].map({1: "Yes", 0: "No"})

#previsões do modelo
preds = pipe_final.predict(inference_clean)
probs = pipe_final.predict_proba(inference_clean)[:, 1]   # prob de ser churn (classe 1)

# adicionar as duas colunas
df_out["churn_yes_no"] = pd.Series(preds).map({1: "Yes", 0: "No"})
df_out["churn_prob"] = probs

prediction.to_csv(f'mahoraga.csv', index=False)